## Asociačné pravidlá - analýza nákupného košíka

Jeden z druhov popisnej analýzy dát sú asociačné pravidlá. Predstavujú špecifický typ popisného modelu, ktorý je aplikovateľný na transakčných dátach. To sú dáta reprezentujúce napr. nákupné, alebo iné transakcie. Riadky v nich zodpovedajú jednotlivých transakciám (napr. jednotlivým nákupom) a stĺpce potom reprezentujú jednotlivé tovary. Hodnoty v jednotlivých stĺpcovh potom indikujú, či v sa v danom nákupe niektorý z tovarov vyskytol, alebo nie.

Asociačné pravidlá potom používame na odhalenie častých kombinácií položiek v rámci množiny transakcií. Tento princíp nám teda pomôže identifikovať napr. položky, ktoré sa často nakupujú spoločne (odtiaľ aj odvodený názov - analýza nákupného košíka).

Pre hľadanie asociačných pravidiel v Pythone potrebujeme najprv doinštalovať moduly obsahujúce potrebné algoritmy pre ich hľadanie. Keďže knižnica Scikit-learn takéto neobsahuje, doinštalujeme modul `mlxtend`, ktorý obsahuje algoritmus Apriori, často používaný pre hľadanie asociačných pravidiel. 

Pre inštaláciu v distribúcii Anaconda je potrebné sa prepnúť do domovskej aplikácie (Anaconda Navigator). V záložke `Environments` potom kliknite na trojuholníkový symbol pri položke `base (root)`. V nej zvoľte možnosť `Open Terminal` a v príkazovom riadku zadajte príkaz `pip install mlxtend`. Takto nainštalujeme modul a budeme ho môcť používať v skriptoch a Jupyter notebookoch. 

In [1]:
import pandas as pd # importujeme pandas
from mlxtend.frequent_patterns import apriori # importujeme knižnice pre apriori algoritmus
from mlxtend.frequent_patterns import association_rules # importujeme knižnicu pre asociačné pravidlá

Ako ukážkový dataset použijeme dataset Online Retail, ktorý obsahuje záznamy z transakcií nákupov rôzneho tovaru prostredníctvom internetového obchodu v rôznych krajinách. Pre demonštračné účely je dataset používaný v tomto notebooku zredukovaný. 

Dáta načítame zo súboru a po vypísaní hlavičky vidíme štruktúru datasetu:
* číslo faktúry
* identifikačné číslo tovaru
* názov tovaru
* množstvo
* dátum nákupu
* cena za jednotku tovaru
* ID zákazníka
* krajina

In [2]:
data = pd.read_excel('../data/retail.xlsx') # načítame dáta zo súboru
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Ako vidíme, dáta nemáme v požadovanej - transakčnej - podobe. Pre tieto účely musíme dáta predspracovať a zmeniť ich štruktúru. Upravíme medzery pre atribút `Description` a vyhodíme riadky, ktoré nemajú valídnu faktúru.

In [3]:
data['Description'] = data['Description'].str.strip() # orežeme nežiadúce medzery na začiatkoch a koncoch popisov
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # odstránime riadky, ktoré majú chýbajúcu faktúru
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
data['InvoiceNo'] = data['InvoiceNo'].astype('str') # zakódujeme číslo faktúry ako string
data = data[~data['InvoiceNo'].str.contains('C')] #nepoužijem všetky faktúry, ktoré obsahujú v názve "C"

Faktúru potom použijeme ako identifikátor transakcie - vstupné dáta transformujeme do dátového rámca `basket`, ktorý bude obsahovať riadky reprezentujúce nákupy (identifikované pomocou čísla faktúry) a jednotlivé atribúty budú reprezentovať počty nakúpených tovarov. Toto realizujeme pomocou zoskupenia (`groupBy`) podľa čísla faktúry, položky a množstva. Pre selekciu dát ešte môžeme zvoliť aj atribút charakterizujúci krajinu, a môžeme tak identifikovať časté kombinácie nákupov na rôznych trhoch.
Výslednú tabuľku si môžeme pozrieť vypísaním hlavičky. 

In [5]:
basket = (data[data['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0) #reset index robí to, že chcem zmeniť dĺžku data frameu (rozmer koľko je faktúr), fillna - ak nie je produkt vo faktúre, vyplní mi 0
          .set_index('InvoiceNo'))
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED RETROSPOT,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,72 SWEETHEART FAIRY CAKE CASES,...,WOODEN CROQUET GARDEN SET,WOODEN SKITTLES GARDEN SET,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND PARTY BAG + STICKER SET,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP RED APPLES,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0


Pre jednoduššiu prácu s asociačnými pravidlami ešte dáta binarizujeme - jednotlivé hodnoty atribútov nám budú iba indikovať, či bola daná položka zakúpená alebo nie (počet nebudeme brať do úvahy). Preto pomocou jednoduchej funkcie transformujeme dáta. Zároveň vyhodíme aj atribút `POSTAGE` popisujúci poštovné. 

In [6]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED RETROSPOT,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,72 SWEETHEART FAIRY CAKE CASES,...,WOODEN CROQUET GARDEN SET,WOODEN SKITTLES GARDEN SET,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND PARTY BAG + STICKER SET,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP RED APPLES,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0


Pre hľadanie asociačných pravidiel najprv identifikujeme tzv. frekventované položky. V tomto prípade to bude predstavovať zoznam často nakupovaných položiek (samostatne, nie spolu), ktorý bude doprevádzaný informáciou o ich podpore (`support`) - tzn. podieli transakcií, v ktorých sa daná položka vyskytuje spomedzi všetkých transakcií. Pri vytváraní zoznamu týchto položie môžeme podporu použiť ako parameter pre orezanie množstva identifikovaných vzorov.

In [12]:
pd.set_option('display.max_colwidth', -1)
frequent_itemsets = apriori(basket_sets, min_support=0.13, use_colnames=True)
frequent_itemsets['Length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) #urobím si ďalší stĺpec s počtom itemsets

frequent_itemsets.head(50)

,support,itemsets,Length
0,0.135135,(ALARM CLOCK BAKELIKE GREEN),1
1,0.135135,(ALARM CLOCK BAKELIKE PINK),1
2,0.135135,(ALARM CLOCK BAKELIKE RED),1
3,0.162162,(BAKING SET 9 PIECE RETROSPOT),1
4,0.135135,(CHARLOTTE BAG DOLLY GIRL DESIGN),1
5,0.135135,(CLOTHES PEGS RETROSPOT PACK 24),1
6,0.135135,(GUMBALL COAT RACK),1
7,0.135135,(JAM MAKING SET PRINTED),1
8,0.189189,(LUNCH BAG RED RETROSPOT),1
9,0.135135,(LUNCH BAG WOODLAND),1


Z frekventovaných položiek potom vygenerujeme asociačné pravidlá v tvare `IF` predpoklady (`antecenteds`) `THEN` závery (`consequents`). K jednotlivým pravidlám potom knižnica vypíše aj podporu daného pravidla (podporu predpokladu, záveru a aj celého pravidla), spoľahlivosť (`confidence`) alebo `lift` pravidla. 

In [14]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=5)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
6,(SET OF 2 TEA TOWELS APPLE AND PEARS),(LUNCH BOX WITH CUTLERY RETROSPOT),0.135135,0.189189,0.135135,1.000000,5.285714,0.109569,inf
7,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF 2 TEA TOWELS APPLE AND PEARS),0.189189,0.135135,0.135135,0.714286,5.285714,0.109569,3.027027
8,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE RED)",(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
9,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE PINK)",(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf


Pri veľkom množstve pravidiel je samozrejme možné prehľadávať a orezávať vygenerované pravidlá pomocou formovania podmienok a kritérií pre zobrazovanie pravidiel. Napr. príklad nižšie demonštruje zobrazenie len tých pravidiel, ktorých podpora je väčšia ako 0.2 a spoľahlivosť väčšia ako 0.9.

In [22]:
rules[ (rules['support'] >= 0.13) & (rules['confidence'] >= 0.7) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
6,(SET OF 2 TEA TOWELS APPLE AND PEARS),(LUNCH BOX WITH CUTLERY RETROSPOT),0.135135,0.189189,0.135135,1.000000,5.285714,0.109569,inf
7,(LUNCH BOX WITH CUTLERY RETROSPOT),(SET OF 2 TEA TOWELS APPLE AND PEARS),0.189189,0.135135,0.135135,0.714286,5.285714,0.109569,3.027027
8,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE RED)",(ALARM CLOCK BAKELIKE PINK),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
9,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKELIKE PINK)",(ALARM CLOCK BAKELIKE RED),0.135135,0.135135,0.135135,1.000000,7.400000,0.116874,inf
